In [1]:
from ml_dl_experiments import settings, show_image_from_source

# **torchvision. Использование моделей и подготовка данных**

Мы познакомились с детекторами Faster R-CNN и FCOS и разобрали их архитектурные различия.

Поняли, что эти модели делают и как они работают в теории. Теперь внаучимся применять эти модели на практике в среде PyTorch. Здесь пригодится библиотека `torchvision`. 

Но есть нюанс — если использовать предобученную модель из библиотеки напрямую, без подготовки данных, неизбежно возникнут ошибки. Модель не умеет работать с сырыми изображениями и аннотациями — она ожидает на вход данные в стандартизированном формате. Этот инженерный аспект часто остаётся за кадром, но критически важен для реального проекта. 

## **Библиотека torchvision**

1. Наборы данных. Это готовые классы для работы с часто используемыми академическими датасетами (torchvision.datasets).
    
2. Модели. Это реализации популярных архитектур нейронных сетей для CV (torchvision.models).
    
3. Трансформации. Это утилиты для предобработки и аугментации изображений (torchvision.transforms).

**torchvision.datasets**

**torchvision.transforms**

**torchvision.models**

**Адаптация датасета**

**Конвертация масок в рамки**

**Функция collate_fn**

## **Сравнение моделей YOLO, FCOS и Faster R-CNN**

На этом шаге важно не просто использовать одну модель, а осмысленно выбирать между разными архитектурами.

Архитектуры детекторов объектов бывают фундаментально разными. Все они балансируют между скоростью и точностью, и каждый подход находит свою точку баланса. Чтобы разобраться, что когда уместно применить, лучше всего сравнить модели на практике! 

Мы пропустим через каждую модель один и тот же набор данных, на практике измерим их производительность, визуально оценим качество предсказаний на сложных примерах и проанализируем их характерные ошибки.

**Теоретический обзор архитектур**

| Метод       | Тип детектора   | Основной принцип работы                                                                 | Преимущества                                      | Недостатки                                              |
|-------------|------------------|------------------------------------------------------------------------------------------|---------------------------------------------------|---------------------------------------------------------|
| **Faster R-CNN** | Двухэтапный      | 1. RPN генерирует предложения регионов.<br>2. Детектор классифицирует и уточняет рамки каждого региона. | Очень высокая точность                            | Большие вычислительные затраты, медленная обработка    |
| **FCOS**        | Однопроходный    | Для каждой точки на изображении предсказывает принадлежность к объекту и расстояния до границ рамки. Без якорей. | Хороший баланс скорости и точности, проще архитектура | Генерирует много кандидатов, требуется NMS              |
| **YOLOv10**     | Однопроходный    | Одновременно определяет объекты и подавляет избыточные предсказания на этапе прямого прохода. | Высокая скорость и эффективность end-to-end       | Ниже точность по сравнению с двухэтапными в сложных сценах |

**Подготовка к эксперименту**

**Особенности определения моделей**

**Скорость моделей детекции**

## **MMdetection — стандарт современного проекта**

Ранее мы на практике сравнили три ключевые архитектуры: двухстадийного Faster R-CNN, одностадийного YOLO и anchor-free FCOS. 

При этом вручную запускали инференс, анализировали результаты и измеряли производительность, чтобы понять различия подходов. Это трудоёмкий процесс. В реальной разработке для каждого эксперимента нужна уникальная кодовая база, так что воспроизводить результаты и тестировать новые модели — трудозатратно. Высокие инженерные затраты — главная проблема экспериментирования. 

Чтобы решить эту проблему, в индустрии и академических кругах разработали унифицированные фреймворки. Они стандартизируют и автоматизируют всю инженерную обвязку, позволяя человеку сосредоточиться на главном: архитектуре, данных и гипотезах. В уроке вы познакомитесь с золотым стандартом в этой области — **MMDetection**.

Сейчас мы перейдём от анализа отдельных моделей к построению полноценных, воспроизводимых проектов. Освоив **MMDetection**, мы получим доступ к десяткам SOTA-архитектур и научимся применять промышленный подход к решению задач компьютерного зрения.

**Проблемы ручного подхода**

Основные проблемы такие: 

- **Огромное количество кода.** Сотни, а иногда тысячи строк кода, который не относится напрямую к исследованию, а является инженерной обвязкой.
    
- **Сложность воспроизведения.** При попытке воспроизвести результаты из статьи метрики могут не совпасть. Почему? Может, неправильно реализован learning rate scheduler? Или аугментации были другими? Возможных причин много.
    
- **Трудность в экспериментах.** Хочется попробовать другой backbone (например, сменить ResNet-50 на VGG)? Придётся переписывать значительную часть кода.

**MMDetection**

**MMDetection — это open-source библиотека**, построенная на PyTorch, которая решает все эти проблемы. Это унифицированный фреймворк, который часто используется для продакшн-решений задач детекции и других. 

Преимущества фреймворка: 

1. Обеспечивает воспроизводимость. Все параметры эксперимента описываются в одном файле конфигурации.
    
2. Содержит зоопарк моделей. В MMDetection есть огромная коллекция уже обученных моделей на стандартных датасетах (COCO и другие). Можно сразу использовать готовую модель, а можно — дообучить её на своих данных, экономя недели на обучение с нуля.
    
3. Обеспечивает модульность и гибкость. Модель представлена как конструктор. Нужно заменить backbone? Просто измените одну строчку в config-файле. Нужно добавить новую аугментацию? Добавьте один блок в пайплайн. Все компоненты взаимозаменяемы.
    
4. Поддерживает SOTA-решения. Команда MMDetection оперативно добавляет реализации новых моделей. Удобно использовать передовые технологии, не реализуя их с нуля.